# RSI Divergence Analysis

This notebook analyzes how the RSI divergence detector identifies trend waves and calculates TP/SL levels.

In [7]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

# Add project path to system path
path_splited = os.path.abspath('.').split('rsi_divergence_detector')[0]
PROJECT_PATH = os.path.join(path_splited, 'rsi_divergence_detector')
sys.path.append(PROJECT_PATH)

# Import project modules
from src.data_process.divergence import DivergenceDetector
from src.data_process.data_preprocess import DataPreprocess
from src.data_process.data_fetcher import DataFetcher

## Load Data

First, let's load the price data and calculate RSI.

In [8]:
# Check if we have saved data
try:
    training_data = pd.read_pickle(f"{PROJECT_PATH}/data/training_data.pickle")
    print("Loaded training data from pickle file")
except FileNotFoundError:
    print("No saved training data found. Loading from CSV files...")
    # Load data from CSV files
    data_fetcher = DataFetcher()
    symbol = 'BTC/USDT'
    timeframe = '5m'  # We'll focus on 5-minute timeframe for this analysis
    
    try:
        df = data_fetcher.load_data(symbol, timeframe)
        print(f"Loaded {timeframe} data from CSV")
    except FileNotFoundError:
        print("No CSV file found. Please run data_fetcher.py first to download data.")
        # You can uncomment the following line to fetch data directly, but it might take time
        # df = data_fetcher.fetch_ohlcv(symbol, timeframe)
        # df.to_csv(f"{PROJECT_PATH}/data/{symbol.replace('/', '_')}_{timeframe}.csv")
        raise
    
    # Preprocess data
    data_preprocessor = DataPreprocess()
    df = data_preprocessor.calculate_rsi(df)
    df['timeframe'] = timeframe
    training_data = df

# Filter to a specific timeframe for analysis
timeframe = '5m'
df = training_data[training_data['timeframe'] == timeframe].copy()

# Display the first few rows
df.head()

No saved training data found. Loading from CSV files...
No CSV file found. Please run data_fetcher.py first to download data.


FileNotFoundError: data/BTC_USDT_5m.csv does not exist. Please fetch the data first.

## Analyze Divergence Detection

Let's create a function to visualize how the divergence detector identifies trend waves and calculates TP/SL levels.

In [ ]:
def visualize_divergence(df, start_date=None, end_date=None, max_lookback_bars=50):
    """Visualize divergence detection and TP/SL calculation."""
    # Filter data by date range if provided
    if start_date and end_date:
        df_subset = df[(df.index >= start_date) & (df.index <= end_date)].copy()
    else:
        # Use the last 200 bars if no date range is provided
        df_subset = df.iloc[-200:].copy()
    
    # Create divergence detector
    detector = DivergenceDetector()
    
    # Find divergences
    divergence_df = detector.find_divergences(
        df_subset, 
        bullish_rsi_threshold=35, 
        bearish_rsi_threshold=65,
        min_bars_lookback=5,
        max_bars_lookback=180
    )
    
    if divergence_df.empty:
        print("No divergences found in the selected time range.")
        return
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), gridspec_kw={'height_ratios': [3, 1]})
    
    # Plot price
    ax1.plot(df_subset.index, df_subset['close'], label='Close Price', color='black', alpha=0.7)
    
    # Plot RSI
    ax2.plot(df_subset.index, df_subset['rsi'], label='RSI', color='blue')
    ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5)
    ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5)
    
    # Format x-axis
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    plt.xticks(rotation=45)
    
    # Analyze each divergence
    for i, (start_dt, row) in enumerate(divergence_df.iterrows()):
        end_dt = row['end_datetime']
        divergence_type = row['divergence']
        is_bullish = divergence_type == 'Bullish Divergence'
        color = 'green' if is_bullish else 'red'
        
        # Find previous peak/valley with both old and new methods
        prev_peak_old = detector.find_previous_peak(
            df_subset, start_dt, is_bullish, 
            bullish_peak_rsi_threshold=55, 
            bearish_peak_rsi_threshold=45
        )
        
        prev_peak_new = detector.find_previous_peak(
            df_subset, start_dt, is_bullish, 
            bullish_peak_rsi_threshold=55, 
            bearish_peak_rsi_threshold=45,
            max_lookback_bars=max_lookback_bars
        )
        
        if prev_peak_old is None or prev_peak_new is None:
            continue
        
        # Calculate TP/SL with old method
        tp_old, sl_old = detector.calculate_tp_sl(prev_peak_old, end_dt, df_subset, is_bullish)
        
        # Calculate TP/SL with new method (using the modified function)
        tp_new, sl_new = detector.calculate_tp_sl(prev_peak_new, end_dt, df_subset, is_bullish)
        
        # Plot divergence points
        ax1.scatter(start_dt, df_subset.loc[start_dt, 'close'], color=color, marker='o', s=100)
        ax1.scatter(end_dt, df_subset.loc[end_dt, 'close'], color=color, marker='o', s=100)
        ax1.plot([start_dt, end_dt], [df_subset.loc[start_dt, 'close'], df_subset.loc[end_dt, 'close']], 
                 color=color, linestyle='--')
        
        # Plot RSI divergence
        ax2.scatter(start_dt, df_subset.loc[start_dt, 'rsi'], color=color, marker='o', s=100)
        ax2.scatter(end_dt, df_subset.loc[end_dt, 'rsi'], color=color, marker='o', s=100)
        ax2.plot([start_dt, end_dt], [df_subset.loc[start_dt, 'rsi'], df_subset.loc[end_dt, 'rsi']], 
                 color=color, linestyle='--')
        
        # Plot previous peak/valley
        ax1.scatter(prev_peak_old, df_subset.loc[prev_peak_old, 'close'], color='purple', marker='*', s=150)
        ax1.scatter(prev_peak_new, df_subset.loc[prev_peak_new, 'close'], color='orange', marker='*', s=150)
        
        # Plot trend wave
        if is_bullish:
            ax1.plot([prev_peak_old, end_dt], 
                     [df_subset.loc[prev_peak_old, 'high'], df_subset.loc[end_dt, 'low']], 
                     color='purple', linestyle='-', alpha=0.5)
            ax1.plot([prev_peak_new, end_dt], 
                     [df_subset.loc[prev_peak_new, 'high'], df_subset.loc[end_dt, 'low']], 
                     color='orange', linestyle='-', alpha=0.5)
        else:
            ax1.plot([prev_peak_old, end_dt], 
                     [df_subset.loc[prev_peak_old, 'low'], df_subset.loc[end_dt, 'high']], 
                     color='purple', linestyle='-', alpha=0.5)
            ax1.plot([prev_peak_new, end_dt], 
                     [df_subset.loc[prev_peak_new, 'low'], df_subset.loc[end_dt, 'high']], 
                     color='orange', linestyle='-', alpha=0.5)
        
        # Plot TP/SL levels
        entry_dt = row['entry_datetime']
        ax1.axhline(y=tp_old, color='purple', linestyle='-.', alpha=0.7)
        ax1.axhline(y=sl_old, color='purple', linestyle=':', alpha=0.7)
        ax1.axhline(y=tp_new, color='orange', linestyle='-.', alpha=0.7)
        ax1.axhline(y=sl_new, color='orange', linestyle=':', alpha=0.7)
        
        # Add annotations
        ax1.annotate(f"Old TP: {tp_old:.2f}", xy=(entry_dt, tp_old), xytext=(10, 10), 
                    textcoords='offset points', color='purple')
        ax1.annotate(f"Old SL: {sl_old:.2f}", xy=(entry_dt, sl_old), xytext=(10, -20), 
                    textcoords='offset points', color='purple')
        ax1.annotate(f"New TP: {tp_new:.2f}", xy=(entry_dt, tp_new), xytext=(10, 30), 
                    textcoords='offset points', color='orange')
        ax1.annotate(f"New SL: {sl_new:.2f}", xy=(entry_dt, sl_new), xytext=(10, -40), 
                    textcoords='offset points', color='orange')
        
        # Print analysis
        print(f"Divergence {i+1}: {divergence_type}")
        print(f"  Start: {start_dt}, End: {end_dt}, Entry: {entry_dt}")
        print(f"  Old method - Previous peak: {prev_peak_old}, TP: {tp_old:.2f}, SL: {sl_old:.2f}")
        print(f"  New method - Previous peak: {prev_peak_new}, TP: {tp_new:.2f}, SL: {sl_new:.2f}")
        print(f"  Old TP/SL ratio: {abs(tp_old - row['entry_price']) / abs(sl_old - row['entry_price']):.2f}")
        print(f"  New TP/SL ratio: {abs(tp_new - row['entry_price']) / abs(sl_new - row['entry_price']):.2f}")
        print()
    
    # Set labels and title
    ax1.set_title('Price Chart with Divergence Analysis')
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True)
    
    ax2.set_title('RSI')
    ax2.set_ylabel('RSI Value')
    ax2.set_ylim(0, 100)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return divergence_df

## Analyze Recent Divergences

Let's analyze some recent divergences to see how the trend wave detection and TP/SL calculation work.

In [ ]:
# Analyze the most recent data
# You can adjust the date range to focus on specific periods
divergence_df = visualize_divergence(df, max_lookback_bars=50)

## Experiment with Different Parameters

Let's try different parameters to see how they affect the trend wave detection and TP/SL calculation.

In [ ]:
# Try with a shorter lookback period
divergence_df = visualize_divergence(df, max_lookback_bars=30)

In [ ]:
# Try with a longer lookback period
divergence_df = visualize_divergence(df, max_lookback_bars=80)

## Analyze Specific Time Periods

You can also analyze specific time periods where you know there are interesting divergences.

In [ ]:
# Example: Analyze a specific time period
# Replace these dates with periods you're interested in
start_date = '2023-01-01'
end_date = '2023-01-15'

try:
    divergence_df = visualize_divergence(df, start_date=start_date, end_date=end_date, max_lookback_bars=50)
except KeyError:
    print(f"No data available for the period {start_date} to {end_date}")

## Summary

This notebook demonstrates how the RSI divergence detector identifies trend waves and calculates TP/SL levels. The key improvements made are:

1. **More Conservative Trend Wave Detection**: Limited the lookback period to avoid selecting peaks that are too far back.
2. **Improved TP/SL Calculation**: Used more conservative Fibonacci levels for TP and added a small buffer to SL.

You can experiment with different parameters to find the optimal settings for your trading strategy.